In [1]:
print('MLOPS-Capstone-Project')

MLOPS-Capstone-Project


In [ ]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [9]:
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /home/mahi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mahi/nltk_data...


True

In [10]:
df=pd.read_csv("data.csv")

In [11]:
df.head()

,review,sentiment
0,Every great gangster movie has under-currents ...,positive
1,"I just saw this film last night, and I have to...",positive
2,This film is mildly entertaining if one neglec...,negative
3,Quentin Tarantino's partner in crime Roger Ava...,negative
4,I sat through this on TV hoping because of the...,negative


In [12]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_131007/3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [13]:
df=normalize_text(df)

In [14]:
df.head()

,review,sentiment
0,every great gangster movie under current human...,positive
1,saw film last night say loved every minute tak...,positive
2,film mildly entertaining one neglect acknowled...,negative
3,quentin tarantino s partner crime roger avary ...,negative
4,sat tv hoping name would worth time but dear g...,negative


In [15]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [ ]:
x=df['sentiment'].isin(['positive','negative'])

df=df[x]

In [19]:
df['sentiment']=df['sentiment'].map({'positive':1,'negative':0})
df.head()

,review,sentiment
0,every great gangster movie under current human...,1
1,saw film last night say loved every minute tak...,1
2,film mildly entertaining one neglect acknowled...,0
3,quentin tarantino s partner crime roger avary ...,0
4,sat tv hoping name would worth time but dear g...,0


In [21]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [36]:
vectorizer=CountVectorizer(max_features=150)
X=vectorizer.fit_transform(df['review'])
y=df['sentiment']

In [37]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=442)

In [38]:
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/Mahi-Anol/MLOPS_CAPSTONE_PROJECT.mlflow')
dagshub.init(repo_owner='Mahi-Anol', repo_name='MLOPS_CAPSTONE_PROJECT', mlflow=True)

2025-06-08 17:07:35,380 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/Mahi-Anol/MLOPS_CAPSTONE_PROJECT "HTTP/1.1 200 OK"


Initialized MLflow to track repo "Mahi-Anol/MLOPS_CAPSTONE_PROJECT"

2025-06-08 17:07:35,388 - INFO - Initialized MLflow to track repo "Mahi-Anol/MLOPS_CAPSTONE_PROJECT"


Repository Mahi-Anol/MLOPS_CAPSTONE_PROJECT initialized!

2025-06-08 17:07:35,390 - INFO - Repository Mahi-Anol/MLOPS_CAPSTONE_PROJECT initialized!


In [39]:
mlflow.set_experiment('LogisticRegression baseline')

<Experiment: artifact_location='mlflow-artifacts:/bdb3f8b24fe6459995f16db78c525732', creation_time=1749379383439, experiment_id='0', last_update_time=1749379383439, lifecycle_stage='active', name='LogisticRegression baseline', tags={}>

In [40]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score

In [41]:
logging.basicConfig(level=logging.INFO,format="%(asctime)s - %(levelname)s - %(message)s")

In [ ]:
logging.info("Starting MLFLOW RUN")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 150)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-06-08 17:07:54,153 - INFO - Starting MLFLOW RUN
2025-06-08 17:07:54,571 - INFO - Logging preprocessing parameters...
2025-06-08 17:07:55,881 - INFO - Initializing Logistic Regression model...
2025-06-08 17:07:55,881 - INFO - Fitting the model...
2025-06-08 17:07:56,104 - INFO - Model training complete.
2025-06-08 17:07:56,105 - INFO - Logging model parameters...
2025-06-08 17:07:56,597 - INFO - Making predictions...
2025-06-08 17:07:56,599 - INFO - Calculating evaluation metrics...
2025-06-08 17:07:56,622 - INFO - Logging evaluation metrics...
2025-06-08 17:07:58,339 - INFO - Saving and logging the model...
2025/06/08 17:08:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
